# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, solo_considerando, solo_resultando, solo_encabezado, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from nlppen.extraccion.utils.extraerFechaRecibido import ExtraerFecha
from pyspark.sql.functions import length


## Configuracion Spark

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "1")
         .config("spark.executor.memory", "6g")
         .config("spark.executor.cores", "1")
         .config("spark.driver.memory", "12g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "64g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

21/11/15 20:25:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://7416269fd645:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [3]:

ejecutar = False
if ejecutar:
    terminos = {
        'seguimiento': [r'\bseguimiento\b'],
        'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
        'se condena': [r'\bse condena\b', r'\bse le condena\b', r'\bse les condena\b'],
        'plan': [r'\bplan\b'],
        'plazo': [r'\bplazo\b']
    }
    seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructuralesNormal')
    print("Cantidad elementos originales : " + str(seleccion.sdf.count()))
    seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True)
    print("Cantidad elementos despues de filtrados : " + str(seleccion.sdf.count()))
    estructurales = SentenciasEstructurales(seleccion)

# Procesamiento de todas las variables de sentencias estructurales

## La siguiente celda ejecuta el cálculo de todas las variables definidas para las sentencias estructurales

Nota: Modificar la bandera "ejecutar" para aplicar todo el procesamiento.

In [4]:
from pyspark.sql.types import *

if ejecutar:
    columnas = {
        'se ordena PER' : ArrayType(StringType()),
        'se ordena LOC' : ArrayType(StringType()),
        'se ordena ORG' : ArrayType(StringType()),
        'se ordena MISC' : ArrayType(StringType()),
        'se ordena GPE' : ArrayType(StringType()),
        'se ordena Ent Pub' : ArrayType(StringType())
    }
    estructurales.separarSeOrdena(columnas, True, True)

    columnas = {
        'se condena PER' : ArrayType(StringType()),
        'se condena LOC' : ArrayType(StringType()),
        'se condena ORG' : ArrayType(StringType()),
        'se condena MISC' : ArrayType(StringType()),
        'se condena GPE' : ArrayType(StringType()),
        'se condena Ent Pub' : ArrayType(StringType())
    }
    estructurales.separarSeCondena(columnas, True, True)

    columnas = {
        'extension sentencia' : IntegerType(),
        'extension por lo tanto' : IntegerType()
    }
    estructurales.extraerExtension(columnas, True)

    columnas = {
        'plazosDefinidos' : ArrayType(TimestampType())
    }
    estructurales.plazosDefinidos(columnas, True)

    columnas = {
        'FechaSolicitud' : TimestampType(),
    }

    estructurales.extrarFechaRecibido(columnas, True)

    columnas = {
        'num resolucion' : StringType()
    }

    estructurales.extraerNumeroSentencia(columnas, True)

    columnas = {
        'inst internacionales' : ArrayType(StringType())
    }

    estructurales.extraerInstrumentosInternacionales(columnas, True)

    columnas = {
        'derechos Norm' : ArrayType(StringType()),
        'derechos GenXPat' : ArrayType(StringType())
    }
    estructurales.extraerDerechos(columnas, True)

    columnas = {
        'derechos Acotados' : ArrayType(StringType()),
        'derechos General' : ArrayType(StringType()),
        'derechos Fundamental' : ArrayType(StringType()),
        'derechos Humano' : ArrayType(StringType())
    }

    estructurales.extraerDerechosSinNormalizar(columnas, True)
    
    columnas = {
        'citasFechas' : MapType(StringType(), TimestampType())
    }
    estructurales.extrarCitaSentenciasFecha(columnas, True)
    #Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
    estructurales.seleccion.guardarDatos()

## Guardar los resultados del preprocesamiento por año

In [5]:
if ejecutar:
    df = spark.read.parquet(seleccion.datasets_path+"/terminos.parquet")
    writer = df.write.partitionBy("anno").mode('Overwrite').parquet('./dataset/procesado')


## Cargar los resultados preprocesados y divididos por año

In [6]:
if not ejecutar:
    seleccion = Seleccion({}, spark, parquet_path='./dataset/procesado', datasets_path='./datasets/estructuralesNormal')
    estructurales = SentenciasEstructurales(seleccion)

## Ejecutar un criterio

In [7]:
print(not ejecutar)

True


In [8]:
if not ejecutar:
    columnas = {
        'citasFechas' : MapType(StringType(), TimestampType())
    }
    estructurales.extrarCitaSentenciasFecha(columnas, True)
    s = estructurales.seleccion.sdf

21/11/15 20:25:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)
s.toPandas()

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,Fechahora_ents,Redactor_ents,CitaSentencias_ents,Reglamento_ents,termino_lst,...,se_condena_Ent_Pub,extension_sentencia,extension_por_lo_tanto,plazosDefinidos,FechaSolicitud,num_resolucion,inst_internacionales,derechos_Norm,derechos_GenXPat,derechos_Acotados,derechos_General,derechos_Fundamental,derechos_Humano,anno,citasFechas
0,2018112102451287_19152.html,*180169790007CO* Exp: 18-016979-0007-CO Res. Nº 2018019152 SALA CONSTITUCION...,SALUD,0.994687,180169790007CO,2018-11-16 09:15:00,Araya García,Recurso de Amparo,Con lugar,False,[las nueve horas quince minutos del dieciseis de noviembre de dos mil diecio...,[Araya García],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],[Con lugar],...,[Caja Costarricense de Seguro Social],18471,2641,[1970-04-01 00:00:00],NaT,2018019152,None,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Salud, Derecho Fun...","[derecho a la salud, derecho fundamental a la salud, derecho a la vida, dere...","[derecho en una parte, derecho a la salud, derecho a la igualdad, derecho a ...","[derecho en una parte de la población, derecho fundamental a la salud, derec...",[derecho fundamental a la salud],[derecho humano que tiene el derecho a la salud de todos los ciudadanos],2018,None
1,2018112063619_17421.html,*180157550007CO* Exp: 18-015755-0007-CO Res. Nº 2018017421 SALA CONSTITUCION...,SALUD,1.000000,180157550007CO,2018-10-19 09:30:00,Fernández Argüello,Recurso de Amparo,Con lugar,False,[las nueve horas treinta minutos del diecinueve de octubre de dos mil diec...,[Fernández Argüello],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],[Con lugar],...,[Caja Costarricense de Seguro Social],17177,2252,None,2018-10-07 00:00:00,2018017421,None,"[Derecho Fundamental a la Salud, Derecho Fundamental a la Salud, Derecho Fun...","[derecho a la salud, derecho a la salud, derecho a la salud, derecho a la sa...","[derecho en una parte, derecho a la salud, derecho a la igualdad, derecho hu...","[derecho a la igualdad de trato ante, derecho en una parte de la población, ...",None,[derecho humano que tiene el derecho a],2018,None
2,201812701524278_20041.html,*180178880007CO* Exp: 18-017888-0007-CO Res. Nº 2018020041 SALA CONSTITUCION...,TRABAJO,0.632804,180178880007CO,2018-11-30 09:30:00,Fernández Argüello,Recurso de Amparo,Con lugar parcial,False,[las nueve horas treinta minutos del treinta de noviembre de dos mil diecioc...,[Fernández Argüello],"[2018-2038, 2016-003772, 2017-018586]",[Reglamento sobre Expediente Electrónico ante el Poder Judicial],"[Con lugar parcial, Sin lugar]",...,[Caja Costarricense de Seguro Social],14300,2130,None,2018-11-09 16:05:00,2018020041,None,"[Derecho Fundamental al Salario, Derecho Fundamental al Salario, Derecho Fun...","[derecho constitucional al salario, derecho constitucional al salario, derec...","[derecho fundamental, derecho constitucional]","[derecho constitucional al salario, derecho fundamental al salario]",[derecho fundamental al salario],None,2018,"{'2017-018586': 2017-11-17 09:00:00, '2018-2038': 2018-02-09 09:00:00, '2016..."
3,201911004458949_21429.html,*180188180007CO* Exp: 18-018818-0007-CO Res. Nº 2018021429 SALA CONSTITUCION...,TRABAJO,0.390122,180188180007CO,2018-12-21 09:20:00,Araya García,Recurso de Amparo,Con lugar,False,[las nueve horas veinte minutos del veintiuno de diciembre de dos mil diecio...,[Araya García],None,[Reglamento sobre Expediente Electrónico ante el Poder Judicial],"[Con lugar, Sin lugar]",...,None,10023,1978,None,2018-11-26 11:28:00,2018021429,None,None,None,None,None,None,None,2018,None
4,20189180242172_14652.html,*180063820007CO* Exp: 18-006382-0007-CO Res. Nº 2018014652 SALA CONSTITUCION...,PETICION,0.086251,180063820007CO,2018-09-07 09:20:00,Fernández Argüello,Recurso de Amparo,Con lugar parcial,True,[las nueve horas veinte minutos del siete de setiembre de dos mil dieciocho .],[Fernánd

In [10]:
from pyspark.sql.functions import explode,posexplode, desc
print("Sentencias con citas (antiguas)", s.filter(s.CitaSentencias_ents.isNotNull()).count())
print("Sentencias con citas (nuevas)", s.filter(s.citasFechas.isNotNull()).count())

print("Citas que no fueron encontradas previamente", s.filter(s.CitaSentencias_ents.isNull() & s.citasFechas.isNotNull()).count())
print("Citas que no fueron encontradas ahora en comparación con las antiguas", s.filter(s.CitaSentencias_ents.isNotNull() & s.citasFechas.isNull()).count())

totalAntiguas = (s.filter(s.CitaSentencias_ents.isNotNull())
    .select(explode('CitaSentencias_ents').alias('CitaSentencias_ents')))
totalNuevas = (s.filter(s.citasFechas.isNotNull())
    .select(explode('citasFechas')))

totalConFecha = totalNuevas.filter(totalNuevas.value.isNotNull())
print("Total de citas (antiguas)", totalAntiguas.count())
print("Total de citas (nuevas)", totalNuevas.count())
print("Total de citas con fecha (nuevas)", totalConFecha.count())
print("Porcentaje de extracción de fechas ", totalConFecha.count()/totalNuevas.count())



Sentencias con citas (antiguas) 348
Sentencias con citas (nuevas) 452


Citas que no fueron encontradas previamente 104
Citas que no fueron encontradas ahora en comparación con las antiguas 0
Total de citas (antiguas) 650
Total de citas (nuevas) 788
Total de citas con fecha (nuevas) 641
Porcentaje de extracción de fechas  0.8134517766497462


In [11]:
totalNuevas.toPandas()
totalConFecha.toPandas()

,key,value
0,2017-018586,2017-11-17 09:00:00
1,2018-2038,2018-02-09 09:00:00
2,2016-003772,2016-03-15 14:00:00
3,2017-009047,2017-06-16 14:00:00
4,2014-003249,2014-03-07 09:00:00
...,...,...
636,1236-94,1994-03-04 09:30:00
637,1147-90,1990-09-21 16:00:00
638,153-90,1990-07-11 16:00:00
639,2663-94,1994-06-07 15:00:00


# Cargar procesamiento de sentencias estructurales.

In [12]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)
totalConFecha.toPandas()

,key,value
0,2017-018586,2017-11-17 09:00:00
1,2018-2038,2018-02-09 09:00:00
2,2016-003772,2016-03-15 14:00:00
3,2017-009047,2017-06-16 14:00:00
4,2014-003249,2014-03-07 09:00:00
5,2002-03851,2002-04-30 14:00:00
6,2009-017232,2009-11-17 08:00:00
7,2012-013384,2012-09-25 14:00:00
8,2008-02545,2008-02-22 08:00:00
9,2001-01465,2001-02-21 14:00:00


# Zona de trabajo (ignorar)

In [13]:
from pyspark.sql.functions import explode, desc
from pyspark.sql import functions as F
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 80)
pd.set_option('display.max_colwidth', 80)
pd.set_option('display.width', None)


In [14]:
ss = (s.filter(s.citasFechas.isNotNull())
    .select(s.expediente, s.num_resolucion, s.anno,explode('citasFechas')))
ss.where(ss.num_resolucion.isNotNull() & ss.key.isNotNull() & ss.value.isNull()).orderBy(ss.num_resolucion).toPandas()

,expediente,num_resolucion,anno,key,value
0,6988V98,07864-98,1998,6560-94,NaT
1,130005150007CO,16854-2012,2013,2012-009453,NaT
2,0504I97,1998-00456,1998,0504-I-97,NaT
3,389F89,1999-03968,1999,227,NaT
4,389F89,1999-03968,1999,65-91,NaT
5,990017860007CO,1999-05746,1999,1095-98,NaT
6,990043500007CO,2000-03818,2000,2000-00161,NaT
7,000004420007CO,2000-05348,2000,2000-03896,NaT
8,000054710007CO,2000-06232,2000,2000-05501,NaT
9,000054870007CO,2000-06553,2000,2000-05501,NaT


In [15]:
s = estructurales.seleccion.sdf
s

In [16]:
import re
def extraerCita(txt, nlp):
    res = []
    doc = nlp(txt)
    mesesIndice = {"enero": 1,
    "febrero": 2 ,
    "marzo": 3,
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7,
    "agosto": 8,
    "septiembre": 9 ,
    "setiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12}
    
    meses = "(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|septiembre|setiembre|octubre|noviembre|diciembre)"
    sentenciaRe = "[Ss]entencia\s*(?:estructural)?\s*"
    numeroRe    = "(?:(?:n[uú]mero)|(?:[nN][ºo°][\.|,]?))"
    sentenciaCitadaRe ="\s*(?P<sentencia_citada>[\d]+\s?\-?[a-z]?\s?\-?[\d]+)[\.|,]?"
    horasRe = "(?:(?:\s?del?\s?las?\s?(?P<hora>[\w|\d]+)?\s?(?:(?:\:|horas?|hrs.?))?\s?)?"
    minutosRe = "(?:(?P<minutos>[\w|\s|\d]+)\s+(?:(?:horas?|hrs.?|minutos?)?))?"
    diaRe = "\s?del?\s?(?:d[íi]a)?(?P<dia>[\w|\s|\d]+)\s+de"
    mesRe = "\s+(?P<mes>"+meses+")+\s+"
    a = "(?:mil\s?novecientos\s?[\w]+\s?y?\s?[\w]+)|(?:dos\s?mil\s?[\w]+)"
    annoRe = "(?:del?)?\s?(?:año)?\s?(?P<anno>(?:(?:[\d][\d][\d][\d])|(?:"+a+")))?)?"
    
    sentenciaCitadaRe = sentenciaRe+numeroRe+sentenciaCitadaRe+horasRe+minutosRe+diaRe+mesRe+annoRe
    
    citaSentenciaExp = re.compile(
            sentenciaCitadaRe,  re.X | re.M | re.I)
    matches = re.finditer(citaSentenciaExp, txt)
    
    sentenciasCitadas = {}
    for match in matches:
        
        start, end = match.span(1)
        startH, endH = match.span(2)
        startMin, endMin = match.span(3)
        startD, endD = match.span(4)
        startM, endM = match.span(5)
        startA, endA = match.span(6)
        
        sentenciaSinEspacios = re.sub("\s+","", doc.text[start : end])
        res.append(sentenciaSinEspacios)
        
        hora = doc.text[startH : endH].strip()
        minutos = doc.text[startMin : endMin].strip()
        dia = doc.text[startD : endD].strip()
        mes = doc.text[startM : endM].strip()
         #Evitar errores como dos mil 2014
        annoNumeroLimpio = re.search("\d+", doc.text[startA : endA])
        anno = None
        if annoNumeroLimpio is not None:
            anno = annoNumeroLimpio.group()
        else:
            anno = doc.text[startA : endA]
            
        convertToNumber = Txt2Numbers()
        horaNum = None
        minNum = None
        diaNum = None
        mesNum = None
        annoNum = None
        if hora.isdigit() == False:
            if hora != '':
                horaNum = convertToNumber.number(hora)
        else:
            horaNum = hora
            
        if minutos.isdigit() == False:
            if minutos != '':
                minNum = convertToNumber.number(minutos)
        else:
            minNum = minutos
            
        if dia.isdigit() == False:
            if dia != '':
                diaNum = convertToNumber.number(dia)
        else:
            diaNum = dia
            
        if anno.strip().isdigit() == False:
            if anno != '':
                ext = ExtraerFecha()
                annoNum = ext.textoAAnno(anno)
        else:
            annoNum = anno
            
        if mes.strip().isdigit() == False:
            if mes != '':
                if mes in mesesIndice:
                    mesNum = str(mesesIndice[mes])
                else:
                    mesNum = None
        else:
            mesNum = mes
        print("--------------")
        t1, t2 = match.span(0)
        print(doc.text[t1 : t2].strip())
        """print(diaNum)
        print(mesNum)
        print(annoNum)
        print(horaNum)
        print(minutos, minNum)
        print("--------------")"""
        fecha = ""
        if diaNum is not None and mesNum is not None and annoNum is not None:
            if horaNum is not None:
                print(minNum is not None)
                if minNum is not None:
                    fecha = str(diaNum)+"/"+str(mesNum)+"/"+str(annoNum)+" "+str(horaNum)+":"+str(minNum)
                else:
                    fecha = str(diaNum)+"/"+str(mesNum)+"/"+str(annoNum)+" "+str(horaNum)+":"+"00"
            else:
                fecha = str(diaNum)+"/"+str(mesNum)+"/"+str(annoNum) +" "+ "00" +":"+"00"
        
        print("FECHA : ", fecha)
        try:
            fechaSentencia = pd.Timestamp(pd.to_datetime(fecha, format='%d/%m/%Y %H:%M')).to_pydatetime()
        except:
            fechaSentencia = None
        
        if sentenciaSinEspacios in sentenciasCitadas:
            if sentenciasCitadas[sentenciaSinEspacios] is None:
                sentenciasCitadas[sentenciaSinEspacios] = fechaSentencia
        else:
            sentenciasCitadas[sentenciaSinEspacios] = fechaSentencia
    print(sentenciasCitadas)

In [17]:
citas = s.where(s.num_resolucion == "2018005814").toPandas()
#citas = s.filter(s.CitaSentencias_ents.isNotNull()).toPandas()
nlp = spark_get_spacy("es_core_news_lg")
for i in range(0, len(citas)):
#for i in range(4, 5):
    sent = citas.iloc[i, 4]
    txt = citas.iloc[i, 1]
    print("******************************")
    print(sent)
    print("Citas")
    print("I: ", i)
    #print(txt)
    #if sent == "170068340007CO":
    #    print(txt)
    print(citas.iloc[i, 12])
    extraerCita(txt.lower(), nlp)

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


******************************
180040550007CO
Citas
I:  0
['2017-1242', '2015-017374']
--------------
sentencia número 2017-1242 de las 9:40 del 27 de enero de 2017
True
FECHA :  27/1/2017 9:40
--------------
sentencia número 2015-017374 de las diez horas diez minutos del seis de noviembre de dos mil quince
False
FECHA :  6/11/2015 10:00
{'2017-1242': datetime.datetime(2017, 1, 27, 9, 40), '2015-017374': datetime.datetime(2015, 11, 6, 10, 0)}


In [18]:
a = None
print(a is not None)
import re

False


In [19]:
txt = "sentencia número 2017-1242 de las 9:40 del 27 de enero de 2017, lo siguiente:"
#txt = "sentencia n° 2002-03851 de las 14:56 horas del 30 de abril de 2002"

meses = "(?:enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|septiembre|setiembre|octubre|noviembre|diciembre)"
sentenciaRe = "[Ss]entencia\s*(?:estructural)?\s*"
numeroRe    = "(?:(?:n[uú]mero)|(?:[nN][ºo°][\.|,]?))"
sentenciaCitadaRe ="\s*(?P<sentencia_citada>[\d]+\s?\-?[a-z]?\s?\-?[\d]+)[\.|,]?"
horasRe = "(?:(?:\s?del?\s?las?\s?(?P<hora>[\w|\d]+)?\s?(?:(?:\:|horas?|hrs.?))?\s?)?"
minutosRe = "(?:(?P<minutos>[\w|\s|\d]+)\s+(?:(?:horas?|hrs.?|minutos?)?))?"
diaRe = "\s?del?\s?(?:d[íi]a)?(?P<dia>[\w|\s|\d]+)\s+de"
mesRe = "\s+(?P<mes>"+meses+")+\s+"
a = "(?:mil\s?novecientos\s?[\w]+\s?y?\s?[\w]+)|(?:dos\s?mil\s?[\w]+)"
annoRe = "(?:del?)?\s?(?:año)?\s?(?P<anno>(?:(?:[\d][\d][\d][\d])|(?:"+a+")))?)?"
    
    
sentenciaCitadaRe = sentenciaRe+numeroRe+sentenciaCitadaRe+horasRe+minutosRe+diaRe+mesRe+annoRe
for match in re.finditer(sentenciaCitadaRe, txt.lower()):
    print(match.group(0))
    """print(match.group(1))
    print(match.group(2))
    print(match.group(3))
    print(match.group(4))
    print(match.group(5))
    print(match.group(6))"""
print(re.findall(sentenciaCitadaRe, txt.lower()))


sentencia número 2017-1242 de las 9:40 del 27 de enero de 2017
[('2017-1242', '9', '40', '27', 'enero', '2017')]
